<a href="https://colab.research.google.com/github/AlexanderF048/GOIT_DS/blob/main/HW_5_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
#---MAIN----------------------------
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

#---PATH----------------------------
from os import listdir
from os.path import isfile, join

from google.colab import drive
drive.mount('/content/gdrive')

#---DATA-ANALISIS-------------------
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import preprocessing

from scipy.fftpack import fft, rfft
from scipy.stats import iqr

#---MODELING------------------------
from sklearn import model_selection
from sklearn import svm

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
path_to_data = 'gdrive/My Drive/Data-science/accelerometer_data/data'
activities_names = listdir(path_to_data)
print(activities_names)

for activity in activities_names:
  print('__________________________________')
  print(activity,':::', len(listdir(join(path_to_data, activity))))

['idle', 'running', 'stairs', 'walking']
__________________________________
idle ::: 879
__________________________________
running ::: 3408
__________________________________
stairs ::: 165
__________________________________
walking ::: 1850


In [31]:
frames_idle = listdir(join(path_to_data, 'idle'))
print(frames_idle[1])
frame = pd.read_csv(join(join(path_to_data, 'idle'), frames_idle[1]))
frame
frame.values.flatten()


idle-211.csv


array([ 2.920930e-01, -9.098000e-02,  9.720459e+00,  2.920930e-01,
       -4.788000e-03,  9.749189e+00,  2.489970e-01, -4.309600e-02,
        9.763555e+00,  2.346320e-01, -6.224900e-02,  9.758766e+00,
        2.346320e-01, -4.309600e-02,  9.777920e+00,  2.394200e-01,
       -1.436500e-02,  9.768343e+00,  2.681510e-01, -1.436500e-02,
        9.787497e+00,  2.681510e-01, -1.915400e-02,  9.739613e+00,
        2.729390e-01, -4.788000e-03,  9.753978e+00,  2.250550e-01,
       -1.436500e-02,  9.763555e+00,  2.346320e-01, -1.436500e-02,
        9.792285e+00,  2.250550e-01, -2.394200e-02,  9.753978e+00,
        2.442090e-01,  4.788000e-03,  9.768343e+00,  2.825160e-01,
       -2.394200e-02,  9.773131e+00,  2.681510e-01, -2.873000e-02,
        9.768343e+00,  2.920930e-01, -4.788400e-02,  9.782708e+00,
        2.585740e-01, -4.788400e-02,  9.782708e+00,  2.202670e-01,
       -5.746100e-02,  9.773131e+00,  3.016690e-01, -3.351900e-02,
        9.739613e+00,  2.777270e-01, -2.873000e-02,  9.768343e

In [9]:
features = [
    'min_val_x',
    'min_val_y',
    'min_val_z',

    'max_val_x',
    'max_val_y',
    'max_val_z',

    'mean_val_x',
    'mean_val_y',
    'mean_val_z',

    #'power',
    #'energy',
    #'entropy',

    'skewness_x',
    'skewness_y',
    'skewness_z',

    'kurtosis_x',
    'kurtosis_y',
    'kurtosis_z',

    'variance_x',
    'variance_y',
    'variance_z',

    'median_x',
    'median_y',
    'median_z',

    'std_x',
    'std_y',
    'std_z',

    'mae_mean_absolute_deviation_x',
    'mae_mean_absolute_deviation_y',
    'mae_mean_absolute_deviation_z',

    'rmse_root_mean_sq_error_x',
    'rmse_root_mean_sq_error_y',
    'rmse_root_mean_sq_error_z',

    'cross_correlation_x_y',
    'cross_correlation_x_z',
    'cross_correlation_y_z',

    #'signal_magnitude_area_x',
    #'signal_magnitude_area_y',
    #'signal_magnitude_area_z',

    #'iqr_interquartile_range_x',
    #'iqr_interquartile_range_y',
    #'iqr_interquartile_range_z'

]

In [27]:
def get_features(frame):
    features = []
    features = np.array(features)

    features = np.concatenate((features, frame.min(axis=0).values), axis=0)
    features = np.concatenate((features, frame.max(axis=0).values), axis=0)
    features = np.concatenate((features, frame.mean(axis=0).values), axis=0)
    features = np.concatenate((features, frame.skew(axis=0).values), axis=0)
    features = np.concatenate((features, frame.kurt(axis=0).values), axis=0)
    features = np.concatenate((features, frame.var(axis=0).values), axis=0)
    features = np.concatenate((features, frame.median(axis=0).values), axis=0)
    features = np.concatenate((features, frame.std(axis=0).values), axis=0)

    frame['mean_x'] = frame.mean(axis=0)['accelerometer_X']
    frame['mean_y'] = frame.mean(axis=0)['accelerometer_Y']
    frame['mean_z'] = frame.mean(axis=0)['accelerometer_Z']

    mae_x = mean_absolute_error(frame['accelerometer_X'], frame['mean_x'])
    mae_y = mean_absolute_error(frame['accelerometer_Y'], frame['mean_y'])
    mae_z = mean_absolute_error(frame['accelerometer_Z'], frame['mean_z'])

    rmse_x = np.sqrt(mean_squared_error(frame['accelerometer_X'], frame['mean_x']))
    rmse_y = np.sqrt(mean_squared_error(frame['accelerometer_Y'], frame['mean_y']))
    rmse_z = np.sqrt(mean_squared_error(frame['accelerometer_Z'], frame['mean_z']))

    metrics = np.array([mae_x, mae_y, mae_z, rmse_x, rmse_y, rmse_z])

    features = np.concatenate((features, metrics), axis=0)

    correlations = frame.corr()
    corr = np.array([correlations['accelerometer_X']['accelerometer_Y'],
                     correlations['accelerometer_X']['accelerometer_Z'],
                     correlations['accelerometer_Y']['accelerometer_Z']])
    
    features = np.concatenate((features, metrics), axis=0)

    signal_magnitude_area = pd.Series(frame['accelerometer_X']).apply(lambda x: np.sum(abs(x)/100)) 
    + pd.Series(frame['accelerometer_Y']).apply(lambda x: np.sum(abs(x)/100)) \
    + pd.Series(frame['accelerometer_Z']).apply(lambda x: np.sum(abs(x)/100))

    features = np.concatenate((features, signal_magnitude_area), axis=0)

    return features 

In [22]:
#test
sma_test = pd.Series(frame['accelerometer_X']).apply(lambda x: np.sum(abs(x)/100)) + pd.Series(frame['accelerometer_Y']).apply(lambda x: np.sum(abs(x)/100)) \
                  + pd.Series(frame['accelerometer_Z']).apply(lambda x: np.sum(abs(x)/100))
np.array(sma_test)

array([0.10103532, 0.1004607 , 0.10055648, 0.10055647, 0.10055648,
       0.10022128, 0.10070013, 0.10026918, 0.10031705, 0.10002975,
       0.10041282, 0.10002975, 0.1001734 , 0.10079589, 0.10065224,
       0.10122685, 0.10089166, 0.10050859, 0.10074801, 0.100748  ,
       0.10036494, 0.10022128, 0.10079589, 0.1007959 , 0.10132262,
       0.10060435, 0.09979033, 0.10026918, 0.09969455, 0.09974245])

In [28]:
get_features(frame)

array([ 1.81959000e-01, -9.09800000e-02,  9.72045900e+00,  2.53625900e-01,
       -2.63362000e-02,  9.76387390e+00,  3.01669000e-01,  3.83070000e-02,
        9.79228500e+00,  2.53625900e-01, -2.63362000e-02,  9.76387390e+00,
        2.53625900e-01, -2.63362000e-02,  9.76387390e+00,  2.53625900e-01,
       -2.63362000e-02,  9.76387390e+00, -6.27109708e-01, -9.40647626e-02,
       -4.70393683e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        3.64158668e-01,  4.96314441e-01, -9.76816725e-02,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  8.51502120e-04,  7.57046866e-04,
        3.01920517e-04,  3.18774611e-33,  1.12069199e-34,  3.26425202e-30,
        2.53785500e-01, -2.39420000e-02,  9.76834300e+00,  2.53625900e-01,
       -2.63362000e-02,  9.76387390e+00,  2.91805092e-02,  2.75144847e-02,
        1.73758602e-02,  5.64601285e-17,  1.05862741e-17,  1.80672411e-15,
        2.34631667e-02,  2.13882133e-02,  1.37053733e-02,  2.86900456e-02,
        2.70520234e-02,  

In [32]:
def class_data_preparation(class_name, class_no):
  path = join(path_to_data, class_name)

  X = []

  for item in listdir(path):
    frame = pd.read_csv(join(path, item))
    frame = frame.values
    frame = frame.flatten()
    X.append(frame)

    y = [class_no]*len(X)

    X = np.array(X)
    y= np.array(y)

    return X, y

In [34]:
def class_statistic_preparation(class_name, class_no):
  path = join(path_to_data, class_name)

  X = []

  for item in listdir(path):
    frame = pd.read_csv(join(path, item))
    features = frame.values
    features = get_features(frame)
    X.append(features)

    y = [class_no]*len(X)

    X = np.array(X)
    y= np.array(y)

    return X, y

In [35]:
def create_dataset(class_preparation):

    X_idle, y_idle = class_preparation('idle', 0)
    X_running, y_running = class_preparation('running', 1)
    X_stairs, y_stairs = class_preparation('stairs', 2)
    X_walking, y_walking = class_preparation('walking', 3)

    X = np.concatenate((X_idle, X_running), axis=0)
    X = np.concatenate((X, X_stairs), axis=0)
    X = np.concatenate((X, X_walking), axis=0)

    Y = np.concatenate((y_idle, y_running), axis=0)
    Y = np.concatenate((Y, y_stairs), axis=0)
    Y = np.concatenate((Y, y_walking), axis=0)

    #['idle', 'running', 'stairs', 'walking']
    return X,Y

In [50]:
X, Y = create_dataset(class_data_preparation)
#print(X)
#print(Y)
X_stat, Y_stat = create_dataset(class_statistic_preparation)
#print(X_stat)
#print(Y_stat)

In [64]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_stat, Y_stat, test_size=0.3)

In [65]:
classifier = svm.SVC(decision_function_shape='ovo', kernel='rbf', gamma=0.005, probability=True)

In [66]:
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', gamma=0.005, probability=True)

In [67]:
y_test

array([2, 0])

In [68]:
y_pred_svm = classifier.decision_function(X_test)
y_pred_svm

array([1.81705548e-08, 2.85720484e-25])

In [73]:
rf_classifier = RandomForestClassifier(n_estimators=15, max_depth=None)

In [74]:
rf_classifier = rf_classifier.fit(X_train, y_train)

In [75]:
y_pred_rf = rf_classifier.predict(X_test)
y_pred_rf

array([1, 1])